# Fine-tuning the DistilBERT Model from HuggingFace

---

In [1]:
from utils import checkGPU
import tensorflow as tf
import torch

c:\ProgramData\anaconda3\envs\python-3.8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkGPU(tensorflow=True), checkGPU(tensorflow=False)

Number of GPUs available with tensorflow: 1
Checking if the GPU is available with PyTorch: True


(None, None)